## Plotting open spaces as polygons

In [1]:
import pandas as pd
import json
import plotly.express as px
px.set_mapbox_access_token(open(".mapbox_token").read())
import plotly.io as pio
pio.renderers.default = "notebook"

In [2]:
with open('data/locations/Open_Space.geojson') as f:
    data = json.load(f)

In [3]:
stations = pd.read_csv('data/locations/bluebike_stations.csv')
open_spaces = pd.read_csv('data/locations/open_spaces.csv')
open_spaces = open_spaces.loc[~open_spaces['Type'].isin(['Cemeteries & Burying Grounds', 'Urban Wilds', 'Open Land', 'Other Open Land'])]
open_spaces_small = open_spaces.loc[open_spaces['Acres'] < 3]

In [ ]:
# version with just chloropleth
fig = px.choropleth_mapbox(open_spaces, geojson=data, locations='OBJECTID', color='Type', featureidkey='properties.OBJECTID', center={'lat': 42.32, "lon": -71.0902}, zoom=10)
fig.update_traces(marker_line_width=0)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
pio.write_json(fig, 'choropleth.json')

In [ ]:
# version with points for small spaces
fig = px.scatter_mapbox(open_spaces_small, lat='Avg. Lat', lon='Avg. Long', color='Type', hover_name='Name', size=[1] * len(open_spaces_small), size_max=3,
                        center={'lat': 42.32, "lon": -71.0902}, zoom=10)

colorscales = [
    ((0.0, '#636EFA'), (1.0, '#636EFA')),
    ((0.0, '#EF553B'), (1.0, '#EF553B')),
    ((0.0, '#00CC96'), (1.0, '#00CC96')),
    ((0.0, '#AB63FA'), (1.0, '#AB63FA')),
    ((0.0, '#00cc96'), (1.0, '#19D3F3')),
    ((0.0, '#FFA15A'), (1.0, '#FFA15A'))
]
for i, space_type in enumerate(open_spaces['Type'].unique()):
    temp = open_spaces[open_spaces['Type'] == space_type]
    fig.add_choroplethmapbox(geojson=data, locations=temp['OBJECTID'],
                             z=[i,] * len(temp), featureidkey="properties.OBJECTID",
                             showlegend=True, text=temp['Name'],
                             colorscale=colorscales[i], showscale=False, marker_line_width=0)

fig.update_layout(mapbox={'accesstoken': open(".mapbox_token").read()}, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
pio.write_json(fig, 'choropleth_with_scatter.json')